In [3]:
import numpy as np
import pandas as pd
import re
import nltk

from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [18]:
df = pd.read_csv('justice.csv',delimiter=',',encoding='utf-8')

In [21]:
df.dropna(inplace=True)
df

,Unnamed: 0,ID,name,href,docket,term,first_party,second_party,facts,facts_len,majority_vote,minority_vote,first_party_winner,decision_type,disposition,issue_area
1,1,50613,Stanley v. Illinois,https://api.oyez.org/cases/1971/70-5014,70-5014,1971,"Peter Stanley, Sr.",Illinois,<p>Joan Stanley had three children with Peter ...,757,5,2,True,majority opinion,reversed/remanded,Civil Rights
2,2,50623,Giglio v. United States,https://api.oyez.org/cases/1971/70-29,70-29,1971,John Giglio,United States,<p>John Giglio was convicted of passing forged...,495,7,0,True,majority opinion,reversed/remanded,Due Process
3,3,50632,Reed v. Reed,https://api.oyez.org/cases/1971/70-4,70-4,1971,Sally Reed,Cecil Reed,"<p>The Idaho Probate Code specified that ""male...",378,7,0,True,majority opinion,reversed/remanded,Civil Rights
4,4,50643,Miller v. California,https://api.oyez.org/cases/1971/70-73,70-73,1971,Marvin Miller,California,"<p>Miller, after conducting a mass mailing cam...",305,5,4,True,majority opinion,vacated/remanded,First Amendment
5,5,50644,Kleindienst v. Mandel,https://api.oyez.org/cases/1971/71-16,71-16,1971,"Richard G. Kleindienst, Attorney General of th...","Ernest E. Mandel, et al.",<p>Ernest E. Mandel was a Belgian professional...,2282,6,3,True,majority opinion,reversed,First Amendment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3297,3297,63322,Yellen v. Confederated Tribes of the Chehalis ...,https://api.oyez.org/cases/2020/20-543,20-543,2020,"Janet L. Yellen, Secretary of the Treasury",Confederated Tribes of the Chehalis Reservatio...,<p>For over a century after the Alaska Purchas...,2340,6,3,True,majority opinion,reversed/remanded,Civil Rights
3298,3298,63324,United States v. Palomar-Santiago,https://api.oyez.org/cases/2020/20-437,20-437,2020,United States,Refugio Palomar-Santiago,"<p>Refugio Palomar-Santiago, a Mexican nationa...",2054,9,0,True,majority opinion,reversed/remanded,Criminal Procedure
3299,3299,63323,Terry v. United States,https://api.oyez.org/cases/2020/20-5904,20-5904,2020,Tarahrick Terry,United States,<p>Tarahrick Terry pleaded guilty to one count...,1027,9,0,False,majority opinion,affirmed,Criminal Procedure
3300,3300,63331,United States v. Cooley,https://api.oyez.org/cases/2020/19-1414,19-1414,2020,United States,Joshua James Cooley,<p>Joshua James Cooley was parked in his picku...,1309,9,0,True,majority opinion,vacated/remanded,Civil Rights


In [22]:
df1 = df.copy()

In [23]:
df1.drop(columns=['Unnamed: 0','ID','docket','name','first_party','second_party','issue_area','facts_len','majority_vote','minority_vote','href','term'],inplace=True)

In [24]:
df1

,facts,first_party_winner,decision_type,disposition
1,<p>Joan Stanley had three children with Peter ...,True,majority opinion,reversed/remanded
2,<p>John Giglio was convicted of passing forged...,True,majority opinion,reversed/remanded
3,"<p>The Idaho Probate Code specified that ""male...",True,majority opinion,reversed/remanded
4,"<p>Miller, after conducting a mass mailing cam...",True,majority opinion,vacated/remanded
5,<p>Ernest E. Mandel was a Belgian professional...,True,majority opinion,reversed
...,...,...,...,...
3297,<p>For over a century after the Alaska Purchas...,True,majority opinion,reversed/remanded
3298,"<p>Refugio Palomar-Santiago, a Mexican nationa...",True,majority opinion,reversed/remanded
3299,<p>Tarahrick Terry pleaded guilty to one count...,False,majority opinion,affirmed
3300,<p>Joshua James Cooley was parked in his picku...,True,majority opinion,vacated/remanded


In [25]:
categorical_data = df1[['decision_type','disposition']]
target_data = df1['first_party_winner']
data = df1['facts']


In [26]:
categorical_data.reset_index(drop=True, inplace=True)
target_data.reset_index(drop=True,inplace=True)
data.reset_index(drop=True,inplace=True)

In [28]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder()
target_data = label_encoder.fit_transform(target_data)

In [31]:
target_data = pd.DataFrame(target_data,columns=['party_first_winner'])


In [33]:
frames = [categorical_data,target_data]
df2 = pd.concat(frames,axis=1,join='inner')
df2

,decision_type,disposition,party_first_winner
0,majority opinion,reversed/remanded,1
1,majority opinion,reversed/remanded,1
2,majority opinion,reversed/remanded,1
3,majority opinion,vacated/remanded,1
4,majority opinion,reversed,1
...,...,...,...
3093,majority opinion,reversed/remanded,1
3094,majority opinion,reversed/remanded,1
3095,majority opinion,affirmed,0
3096,majority opinion,vacated/remanded,1


In [35]:
data = pd.DataFrame(data,columns=['facts'])

In [37]:
data['facts'] = data['facts'].str.replace(r'<[^<>]*>', '', regex=True)
data

,facts
0,Joan Stanley had three children with Peter Sta...
1,John Giglio was convicted of passing forged mo...
2,"The Idaho Probate Code specified that ""males m..."
3,"Miller, after conducting a mass mailing campai..."
4,Ernest E. Mandel was a Belgian professional jo...
...,...
3093,For over a century after the Alaska Purchase i...
3094,"Refugio Palomar-Santiago, a Mexican national, ..."
3095,Tarahrick Terry pleaded guilty to one count of...
3096,Joshua James Cooley was parked in his pickup t...


In [38]:
corpus = data['facts']
token_list = nltk.tokenize.word_tokenize(corpus.str.cat(sep=" "))

In [40]:
ps = nltk.stem.porter.PorterStemmer()
lem = nltk.stem.wordnet.WordNetLemmatizer()

In [41]:
def preprocessing_text(text,stem=False,lemma=True,stopword_list=None):
    text = re.sub(r'[^\w\s]','',str(text).lower().strip())
    
    text_list = text.split()
    if stopword_list is not None:
        text_list = [word for word in text_list if w not in stopword_list]
        
    if stem == True:
        text_list = [ps.stem(word) for word in text_list]
        
    if lemma == True:
        text_list = [lem.lemmatize(word) for word in text_list]
    
    text = " ".join(text_list)
    
    return text

In [42]:
data['cleaned_facts'] = data['facts'].apply(lambda x: preprocessing_text(x))

In [45]:
data.head()

,facts,cleaned_facts
0,Joan Stanley had three children with Peter Sta...,joan stanley had three child with peter stanle...
1,John Giglio was convicted of passing forged mo...,john giglio wa convicted of passing forged mon...
2,"The Idaho Probate Code specified that ""males m...",the idaho probate code specified that male mus...
3,"Miller, after conducting a mass mailing campai...",miller after conducting a mass mailing campaig...
4,Ernest E. Mandel was a Belgian professional jo...,ernest e mandel wa a belgian professional jour...


In [46]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

In [47]:
vectorizer = CountVectorizer()
x = vectorizer.fit_transform(data['cleaned_facts'])

In [48]:
x = x.toarray()

In [50]:
df.reset_index(drop=True, inplace=True)

In [53]:
data = pd.concat([data,target_data['party_first_winner']],axis=1, join='inner')

In [56]:
features = data['cleaned_facts']
target = data['party_first_winner']

In [59]:
X_train, X_test, y_train, y_test = train_test_split(features,target, test_size=0.25)

In [60]:
data_pipeline = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression(solver='liblinear'))])

In [62]:
data_pipeline.fit(X_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()),
                ('lr', LogisticRegression(solver='liblinear'))])

In [63]:
data_pipeline.score(X_test,y_test)

0.5987096774193549

In [64]:
from sklearn.ensemble import RandomForestClassifier


In [65]:
data_pipeline1 = Pipeline(steps=[('cv',CountVectorizer()),('rf',RandomForestClassifier())])


In [66]:
data_pipeline1.fit(X_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()), ('rf', RandomForestClassifier())])

In [67]:
data_pipeline1.score(X_test,y_test)

0.6541935483870968

In [68]:
data_pipeline2= Pipeline(steps=[('cv',CountVectorizer()),('rf',KNeighborsClassifier(n_neighbors=3))])

In [69]:
data_pipeline2.fit(X_train,y_train)

Pipeline(steps=[('cv', CountVectorizer()),
                ('rf', KNeighborsClassifier(n_neighbors=3))])

In [70]:
data_pipeline2.score(X_test,y_test)

0.6348387096774194

In [71]:
df3 = pd.concat([data,categorical_data],axis=1,join='inner')

In [72]:
df3

,facts,cleaned_facts,party_first_winner,decision_type,disposition
0,Joan Stanley had three children with Peter Sta...,joan stanley had three child with peter stanle...,1,majority opinion,reversed/remanded
1,John Giglio was convicted of passing forged mo...,john giglio wa convicted of passing forged mon...,1,majority opinion,reversed/remanded
2,"The Idaho Probate Code specified that ""males m...",the idaho probate code specified that male mus...,1,majority opinion,reversed/remanded
3,"Miller, after conducting a mass mailing campai...",miller after conducting a mass mailing campaig...,1,majority opinion,vacated/remanded
4,Ernest E. Mandel was a Belgian professional jo...,ernest e mandel wa a belgian professional jour...,1,majority opinion,reversed
...,...,...,...,...,...
3093,For over a century after the Alaska Purchase i...,for over a century after the alaska purchase i...,1,majority opinion,reversed/remanded
3094,"Refugio Palomar-Santiago, a Mexican national, ...",refugio palomarsantiago a mexican national wa ...,1,majority opinion,reversed/remanded
3095,Tarahrick Terry pleaded guilty to one count of...,tarahrick terry pleaded guilty to one count of...,0,majority opinion,affirmed
3096,Joshua James Cooley was parked in his pickup t...,joshua james cooley wa parked in his pickup tr...,1,majority opinion,vacated/remanded


In [73]:
features_1 = df3[['cleaned_facts','decision_type','disposition']]
target_1 = df3['party_first_winner']


In [74]:
categorical_data_1 = pd.get_dummies(categorical_data['decision_type'])
categorical_data_2 = pd.get_dummies(categorical_data['disposition'])
categorical_data_3 = pd.concat([categorical_data_1,categorical_data_2],axis=1,join='inner')
categorical_data_3 = pd.concat([categorical_data_3,df3['party_first_winner']],axis=1,join='inner')

In [75]:
categorical_data_3

,dismissal - improvidently granted,dismissal - moot,dismissal - other,equally divided,majority opinion,opinion of the court,per curiam,plurality opinion,affirmed,none,reversed,reversed in-part,reversed in-part/remanded,reversed/remanded,vacated,vacated in-part/remanded,vacated/remanded,party_first_winner
0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
4,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
3094,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
3095,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
3096,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1


In [84]:
count_matrix = vectorizer.fit_transform(df3['cleaned_facts'])
count_array = count_matrix.toarray()
df4 = pd.DataFrame(data=count_array,columns = vectorizer.get_feature_names())

In [86]:
df4 = pd.concat([df4,categorical_data_3],axis=1,join='inner')

In [88]:
X = df4.drop(columns=['party_first_winner'])
y = df4['party_first_winner']

In [90]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=10)

In [91]:
from sklearn.decomposition import PCA
pca = PCA(n_components=400)
model_fit = pca.fit_transform(X_train)

In [92]:
print(pca.explained_variance_ratio_.sum())


0.8081614057058941


In [94]:
pca_df = pd.DataFrame(data = model_fit)
pca_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
0,26.961864,-3.681123,-2.560743,-0.391641,4.451147,0.128970,-3.792748,-0.372157,-0.120997,-2.134441,...,-0.353679,0.178927,-0.642891,-0.531599,-0.672820,-0.464090,-0.096820,0.193147,0.569860,-0.890536
1,0.106509,6.547425,9.470134,-1.296674,-2.169822,-1.738704,-0.046669,-0.885060,-0.967551,1.275347,...,-0.157154,0.168137,-0.025944,-0.111565,0.229469,0.020172,-0.594448,-0.086360,0.254911,-0.725527
2,-6.122700,-3.645523,0.031245,-0.733854,1.437107,-2.148683,-0.175680,-1.195032,-0.959502,-1.038414,...,-0.054894,0.304149,0.158952,-0.028902,0.207842,-0.170102,-0.197575,0.245303,-0.428211,0.416362


In [95]:
final =pd.concat([pca_df,y],axis=1,join='inner')

In [99]:
x=final.drop(columns=['party_first_winner'])
Y=final[['party_first_winner']]

In [100]:
model = RandomForestClassifier()
model.fit(x,Y)


<ipython-input-100-7616b94b0067>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x,Y)


RandomForestClassifier()

In [101]:
pca_fit = pca.fit_transform(X_test)
X_test = pd.DataFrame(data=pca_fit)

In [102]:
model.score(X_test,y_test)

0.6623655913978495

In [103]:
from sklearn.model_selection import GridSearchCV
param_grid = {  'bootstrap': [True], 'max_depth': [5, 10, None], 'max_features': ['auto', 'log2'], 'n_estimators': [5, 6, 7, 8, 9, 10, 11, 12, 13, 15]}

In [104]:
model_1 = RandomForestClassifier(random_state = 1)
g_search = GridSearchCV(estimator = model_1, param_grid = param_grid,
                        cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)

In [105]:
g_search.fit(x, Y);
print(g_search.best_params_)

/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/mod

/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/mod

/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/mod

/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/mod

/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/mod

/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/mod

/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/mod

{'bootstrap': True, 'max_depth': 5, 'max_features': 'log2', 'n_estimators': 13}


/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:593: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


In [106]:
model_1 = RandomForestClassifier(bootstrap=True,max_depth=5,max_features='log2', n_estimators=15)

In [107]:
model_1.fit(x,Y)

<ipython-input-107-dc6c1c62bc59>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_1.fit(x,Y)


RandomForestClassifier(max_depth=5, max_features='log2', n_estimators=15)

In [109]:
y_pred = model_1.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 66.45%


In [139]:
model = XGBClassifier()

param_grid = {'booster' : ['gbtree','gblinear','dart'], 'max_depth' : [1,2,3,4,5], 'n_jobs' : [1,2,3,4,5]}
g_search = GridSearchCV(estimator = model, param_grid = param_grid,
                        cv = 3, n_jobs = 1, verbose = 0, return_train_score=True)
g_search.fit(x,Y)
print(g_search.best_params_)


/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[16:34:52] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[16:34:54] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

[16:44:04] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[16:44:08] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[16:44:11] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
{'booster': 'gbtree', 'max_depth': 1, 'n_jobs': 1}


In [144]:
model = XGBClassifier(booster='gbtree',max_depth=1,n_jobs=1)
model.fit(x, Y)
y_pred1 = model.predict(X_test)
predictions = [round(value) for value in y_pred1]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[16:48:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Accuracy: 65.16%


In [158]:
knn=KNeighborsClassifier(n_neighbors=30)
knn.fit(x,Y)
y_pred = knn.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 64.52%


/Users/apoorvareddy/opt/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
